In [4]:
import yfinance as yf

In [25]:
msft = yf.Ticker("MSFT")
msft.history(period="1m")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-06-19,198.59,199.29,194.37,195.15,44423500,0,0
2020-06-19,198.59,199.29,194.37,195.15,44441141,0,0
